In [526]:
# Import
import itertools
import json

import numpy as np
import random
from munkres import Munkres
import time
from pymoo.indicators.hv import HV


In [527]:
def get_sols(r):
    n=int(r.readline())
    for _ in range(n):
        sols.append(list(map(int, r.readline().split()))[1:])
    return sols

def load_datas(filename: object, N_obj: object, get_sol=True) -> object:
    objectifs = []
    with open(filename, 'r') as r:
        dim = int(r.readline())
        for _ in range(4):
            tmp = [list(map(int, r.readline().split())) for _ in range(dim)]
            objectifs.append(np.array(tmp))

        sols=[]
        if get_sol:
            sols = get_sols(r)
    return objectifs[:N_obj], dim, sols

In [528]:
def create_mat(positions, N):
    vals = []
    tmp = np.zeros((N, N))
    for index in positions:
        tmp[index[0]][index[1]] = 1
    vals.append(tmp)
    return vals

In [529]:
def fast_compute_objectifs(coordinates, objectifs):
    return [int(sum([objectif[x, y] for x, y in coordinates])) for objectif in objectifs]

In [530]:
def sol_is_valid(coordinates):
    rows = set([x for x, y in coordinates])
    columns = set([y for x, y in coordinates])
    if len(rows) == len(coordinates) and len(columns) == len(coordinates):
        # print("Il n'y a pas de doublons.")
        return True
    else:
        # print("Il y a des doublons.")
        return False


In [531]:
def generate_identity_and_reverse(dim):
    identi = np.identity(dim)
    indices = np.nonzero(identi)
    start = list(zip(indices[0], indices[1]))
    flip_start = np.fliplr(identi)
    indices = np.nonzero(flip_start)
    flip_start = list(zip(indices[0], indices[1]))
    return [start, flip_start]

In [532]:
def generate_best_single_obj(objectifs):
    vals = []
    objs_copy = np.copy(objectifs)
    for i in range(len(objectifs)):
        m = Munkres()
        indexes = m.compute(objs_copy[i])
        vals.append(indexes)
    return vals

In [533]:
def write_solutions(lap, filename, A):
    with open(f'solutions_multi/lap-{lap}/{filename}.txt', 'w') as w:
        w.write(f'{len(A)}\n')
        for a in A:
            w.write(' '.join(str(p) for p in a[0]))
            w.write('\n')

    ##########################################################

    with open(f'solutions_pts_multi/lap-{lap}/{filename}.txt', 'w') as w:
        w.write(f'{len(A)}\n')
        for a in A:
            w.write('   '.join(str(p) for p in a[1]))
            w.write('\n')

In [534]:
def make_n_permute(tuple_list, n):
    coordinates = tuple_list.copy()
    samples = random.sample(coordinates, n)
    l1, l2 = zip(*samples)
    line_permute = [(x, y) for x, y in zip(random.sample(l1, len(l1)), l2)]
    for sample in samples:
        coordinates.remove(sample)
    return coordinates + line_permute

In [535]:
import itertools
def permute_all_possibility(tuple_list):
    l1, l2 = zip(*tuple_list)
    possibilities = list(itertools.permutations(l2))
    pts = []
    for p in possibilities:
        pts.append(list(zip(l1, p)))
    return pts

In [536]:
def fast_random_permute(tuple_list, N, exclusions):
    voisins = []
    for i in range(N):
        x, y = zip(*tuple_list)
        random.shuffle(list(y))
        l1 = list(zip(x, y))
        l1.sort()
        # if sol_is_valid(l1) and str(l1) not in exclusions:
        if sol_is_valid(l1):
            voisins.append(l1)
            exclusions.add(str(l1))

        for it in [2, 2, 3,  5, 7,10, 15]:
        # for it in [2, 2, 2, 3, 3, 4, 5]:
            l2 = make_n_permute(tuple_list, it)
            l2.sort()
            # if sol_is_valid(l2) and str(l2) not in exclusions:
            if sol_is_valid(l2):
                voisins.append(l2)
                exclusions.add(str(l2))
        # l2 = make_n_permute(tuple_list, 5)
        # l2.sort()
        # if sol_is_valid(l2) and str(l2) not in exclusions:
        #     voisins.append(l2)
        #     exclusions.add(str(l2))
        # l2 = make_n_permute(tuple_list, 2)
        # l2.sort()
        # if sol_is_valid(l2) and str(l2) not in exclusions:
        #     voisins.append(l2)
        #     exclusions.add(str(l2))
    # voisins+= permute_all_possibility(tuple_list)
    return voisins, exclusions

In [537]:
def update(A, Pa, obj_voisin, voisin, N_obj, exclusions):
    bad_memory=[]
    global_counter=0
    for i in range(len(A)):
        A_grand=0
        A_petit=0
        for k in range(N_obj):
            if A[i][0][k]>=obj_voisin[k]:
                A_grand+=1
            if A[i][0][k]<=obj_voisin[k]:
                A_petit+=1
        if A_petit == N_obj:
            break
        if A_grand == N_obj:
            bad_memory.append(i)
        global_counter+=1
    for indice in sorted(bad_memory, reverse=True):
        A.pop(indice)
    if global_counter == len(A)-len(bad_memory):
        Pa.append(voisin)
        A.append((obj_voisin, voisin))
        # exclusions.add(str(voisin))
    return A, Pa, exclusions

In [538]:
def load_solutions(filename):
    with open(f"solutions_pts_multi/{filename}", 'r') as r:
        N = int(r.readline())
        solutions = []
        for _ in range(N):
            lines = [eval(t) for t in r.readline().split('  ')]
            solutions.append(lines)
    return solutions

In [539]:
def write_exclusions(filename, exclusions):
    with open( "data.txt", 'w') as w:
        w.write(f"{len(exclusions)}\n")
        for pt in exclusions:
            w.write(f"{pt}\n")
def load_exclusions(filename):
    t=set()
    with open( "data.txt", 'r') as r:
        N = int(r.readline())
        for _ in range(N):
            t.add(r.readline().rstrip('\n'))
    return t

In [540]:
def initialise(objectifs, dim, N_obj,exclusions, init = 1 ,filename=''):
    if filename:
        P_temp = load_solutions(filename)
    else:
        P_temp = generate_best_single_obj(objectifs) + generate_identity_and_reverse(dim)
    vals = []
    for val in P_temp:
        vals += fast_random_permute(val, init, exclusions)[0]
    P_temp += vals
    A = []  # archive val obj
    Pa = []
    for p in P_temp:
        obj_voisin = fast_compute_objectifs(p, objectifs)
        if len(A) == 0:
            A.append((obj_voisin, p))
            Pa.append(p)
            p.sort()
            exclusions.add(str(p))
            pass
        A, Pa, exclusions = update(A, Pa, obj_voisin, p, N_obj, exclusions)
    P = Pa
    Pa = []
    return A, P, Pa, exclusions

In [541]:
def pareto_search(A, P, Pa, max_time, exclusions):
    start = time.time()
    stopper = 0
    while P:
        i_start = time.time()
        print(f"Iterations: {stopper + 1}")
        print(f"Lenght of A: {len(A)}")
        print(f"Lenght of P: {len(P)}")
        print(f"*** Excluded pts: {len(exclusions)} ***")
        for x in P:
            voisinage, exclusions = fast_random_permute(x, determine_voisin(P), exclusions)
            for voisin in voisinage:
                obj_voisin = fast_compute_objectifs(voisin, objectifs)
                A, Pa, exclusions = update(A, Pa, obj_voisin, voisin, N_obj, exclusions)
        i_end = time.time()
        print(f"Iteration time: {i_end - i_start}")
        print("----------------------------------")
        P = Pa
        Pa = []
        if (i_end - start) >= max_time:
            break
        stopper += 1
    end = time.time()
    run_time = end-start
    print(f"Total time: {run_time} seconds")
    print(f"Lenght of A: {len(A)}")
    return A, P, Pa, run_time, exclusions

In [542]:
def determine_voisin(P):
    if len(P)<10:
        return 200
    elif len(P)<50:
        return 100
    elif len(P)<100:
        return 50
    elif len(P)<500:
        return 25
    elif len(P)<1000:
        return 10
    elif len(P)<5000:
        return 5
    elif len(P)<10000:
        return 3
    else:
        return 2

In [543]:
N_obj = 4
lap = 30
sols_filename = f'lap-{lap}/_hv_LAP-{lap}.txt'
# sols_filename = f'lap-{lap}/_hv_LAP-{lap}-4obj.txt'
objectifs, dim, sols = load_datas(f'datas/LAP-{lap}.dat', N_obj, get_sol=False)

In [544]:
for _ in range(1):

    max_run_time=int(1*60)
    exclusions = set()
    # exclusions = load_exclusions(f"exclusions_lap-{lap}")
    while max_run_time>0:
        print(f"==== Time remaining: {max_run_time}s ====")
        print(f"*** Excluded pts: {len(exclusions)} ***")
        A, P, Pa, exclusions = initialise(objectifs, dim, N_obj,exclusions,  init = 1,filename=sols_filename)
        A, P, Pa, run_time, exclusions =pareto_search(A, P, Pa, max_run_time, exclusions)
        write_solutions(lap, f"_hv_LAP-{lap}", A)
        max_run_time -=run_time
    write_exclusions(f'exclusion    s_lap-{lap}', exclusions)

==== Time remaining: 900s ====
*** Excluded pts: 0 ***
Iterations: 1
Lenght of A: 10891
Lenght of P: 11278
*** Excluded pts: 72850 ***
Iteration time: 250.1728503704071
----------------------------------
Iterations: 2
Lenght of A: 11349
Lenght of P: 1519
*** Excluded pts: 203756 ***
Iteration time: 131.91725420951843
----------------------------------
Iterations: 3
Lenght of A: 11422
Lenght of P: 306
*** Excluded pts: 247807 ***
Iteration time: 129.06844282150269
----------------------------------
Iterations: 4
Lenght of A: 11443
Lenght of P: 215
*** Excluded pts: 291744 ***
Iteration time: 88.45748949050903
----------------------------------
Iterations: 5
Lenght of A: 11406
Lenght of P: 157
*** Excluded pts: 322488 ***
Iteration time: 63.646722078323364
----------------------------------
Iterations: 6
Lenght of A: 11358
Lenght of P: 109
*** Excluded pts: 345017 ***
Iteration time: 44.47983908653259
----------------------------------
Iterations: 7
Lenght of A: 11349
Lenght of P: 86
***

In [545]:
# pareto = []
# for a in A:
#     pareto.append(np.array(a[0]))
# pareto = np.array(pareto)
# ref_15 = np.array([112, 206, 247, 418])
# ref_20 = np.array([137, 304, 414, 531])
# ref_30 = np.array([211, 381, 581, 835])
# if lap == 15:
#     ref_point = ref_15
# elif lap == 20:
#     ref_point = ref_20
# elif lap == 30:
#     ref_point = ref_30
# else:
#     pt = 900
#     ref_point = np.array([pt, pt, pt, pt])
#
# ind = HV(ref_point=ref_point)
# hv = ind(pareto)
# print("HV", hv)
# print(f"Norm: {hv / (np.prod(ref_point))}")

In [546]:
# write_solutions(lap, f"{int(round(hv / (np.prod(ref_point)) * 100, 0))}_hv_LAP-{lap}", A)
# write_solutions(lap, f"_hv_LAP-{lap}", A)

In [547]:
write_exclusions(f'exclusions_lap-{lap}', set())